In [87]:
#defining imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
from tensorflow.keras import datasets
from keras.layers import Dense
from keras.layers import Conv2D, Layer
from keras import backend as K

In [88]:
#defining training and testing datasets
(x_train, y_train), (x_test,y_test) = datasets.mnist.load_data()

In [89]:
#adding 4th dimension as 1 to declare as grayscale image
#normalizing the images
x_train = x_train.reshape((60000, 28, 28, 1))
x_train = x_train / 255.0
x_test = x_test.reshape((10000, 28, 28, 1))
x_test = x_test / 255.0

Defining our model

In [90]:
# Define the CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), name='conv2d_1'),
    #tf.keras.layers.Conv2D(32, (3, 3), activation='relu', name='conv2d_2'),
    #tf.keras.layers.Conv2D(32, (3, 3), activation='relu', name='conv2d_3'),
    tf.keras.layers.Flatten(name='flatten'),
    #tf.keras.layers.Dense(10, activation='softmax', name='output')
])

In [91]:
#determining the weight and bias shape of convolutional layers
convW,convB = model.layers[0].get_weights()

convW.shape


(3, 3, 1, 32)

In [92]:
# compile the model
opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [93]:
#extracting initial filter matrices

filter_matrices = []
for layer in model.layers:
    if isinstance(layer, Conv2D):
        weights, biases = layer.get_weights()
        filter_matrices.append(weights)
        
filter_matrices[0]

array([[[[-0.01900043,  0.03672551,  0.0430087 , -0.13929585,
           0.02883182,  0.04263088,  0.01293938, -0.03313406,
           0.07048923, -0.05301746,  0.10666698, -0.02829881,
          -0.07272732, -0.03026661,  0.02928135, -0.14199668,
          -0.0850064 , -0.07997881,  0.08467892,  0.06201228,
           0.08452182, -0.01717522,  0.1161408 , -0.0833247 ,
           0.08158541,  0.10179327,  0.12613453,  0.02084211,
          -0.10489421,  0.08390069,  0.01112579,  0.05707273]],

        [[ 0.07412748, -0.04612972, -0.11925267, -0.11221708,
           0.12124406,  0.05095209, -0.06402098, -0.10834963,
           0.05150186,  0.14169027,  0.02309148, -0.02917321,
           0.03705564,  0.07582906,  0.08366331, -0.02029391,
           0.03072931, -0.03979641,  0.14169805, -0.05382916,
           0.01708972, -0.04153989,  0.06078301, -0.1278096 ,
          -0.05652642,  0.00462346, -0.04426724, -0.11840563,
           0.05927421, -0.02479486, -0.12925896,  0.02869645]],

  

In [94]:
# Train the model and display the activations after each epoch
start_time = time.time()
history = model.fit(x_train, y_train, epochs=1, validation_data=(x_test, y_test))
end_time = time.time()

1875/1875 [==============================] - 13s 7ms/step - loss: 10.2937 - accuracy: 0.0955 - val_loss: 9.9819 - val_accuracy: 0.0980


In [95]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print('Test accuracy:', test_acc)
print('Time elapsed: ', end_time - start_time)

313/313 - 1s - loss: 9.9819 - accuracy: 0.0980 - 1s/epoch - 3ms/step
Test accuracy: 0.09799999743700027
Time elapsed:  13.121060132980347


Filter Matrix Extraction and Filter Activations


In [96]:
filter_matrices = []
filter_activations = [] 
for layer in model.layers:
    if isinstance(layer, Conv2D):
        weights, biases = layer.get_weights()
        filter_matrices.append(weights)
        
        activation_func = K.function([model.input], [layer.output])
        activations = np.mean(activation_func([x_train]), axis=(0, 1, 2))
        filter_activations.append(activations)

In [97]:
filter_matrices


[array([[[[-5.86104728e-02, -1.10076042e-02, -1.27518103e-02,
           -1.81280479e-01, -4.90751192e-02, -7.04282802e-03,
           -3.01601347e-02, -9.97914672e-02, -5.92099363e-03,
           -1.18865274e-01,  2.49268301e-02, -4.79805656e-02,
           -1.04117543e-01, -7.06270114e-02, -2.81581152e-02,
           -1.96675867e-01, -1.09621972e-01, -1.01126775e-01,
           -1.34170363e-02,  1.57254543e-02,  2.89389417e-02,
           -4.50095050e-02,  3.28611508e-02, -1.12521715e-01,
            3.67065221e-02,  4.34907386e-03,  5.07714115e-02,
           -1.96193904e-02, -1.19913779e-01,  3.01126707e-02,
           -6.38457900e-03, -2.14626566e-02]],
 
         [[ 3.39477360e-02, -8.87670740e-02, -1.53255105e-01,
           -1.59266099e-01,  4.49395776e-02, -1.28322106e-03,
           -1.23752557e-01, -1.81632310e-01, -2.17473432e-02,
            4.43320945e-02, -5.91279194e-02, -6.34492040e-02,
            2.69537471e-04,  2.36512963e-02,  2.69017126e-02,
           -5.6588631

In [98]:
filter_activations

[array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [99]:
# Step 3: Rank Filters based on their mean of activation values in the feature map

filter_ranks = np.argsort(filter_activations)
filter_ranks

array([[[ 0, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15,
         14, 13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1, 30, 31],
        [ 0, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15,
         14, 13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1, 30, 31],
        [ 0, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15,
         14, 13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1, 30, 31],
        [ 0, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15,
         14, 13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1, 30, 31],
        [ 0, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15,
         14, 13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1, 30, 31],
        [ 0, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15,
         14, 13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1, 30, 31],
        [ 0, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15,
         14, 13, 12, 11, 10,  9,  8,  7,  6, 

In [100]:
# Step 4: remove Filters
num_filters_to_keep = 32  # Number of filters to keep
        
if isinstance(model.layers[0], tf.keras.layers.Conv2D):
    weights, biases = model.layers[0].get_weights()
    keep_weights = weights[:, :, :, filter_ranks[-num_filters_to_keep:]]
    keep_weights = keep_weights[:,:,:,0,0,:]
    keep_biases = biases[filter_ranks[-num_filters_to_keep:]]
    keep_biases=keep_biases[0,0,:]
    model.layers[0].set_weights([keep_weights, keep_biases])


In [101]:
#evaluate test dataset after filter manipulation
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print('Test accuracy:', test_acc)
print('Time elapsed: ', end_time - start_time)

313/313 - 1s - loss: 9.9819 - accuracy: 0.0980 - 964ms/epoch - 3ms/step
Test accuracy: 0.09799999743700027
Time elapsed:  13.121060132980347
